
# **Neural machine translation with attention**


This notebook trains a sequence to sequence (seq2seq) model for English to Deutch translation. This is an advanced example that assumes some knowledge of sequence to sequence models.

After training the model in this notebook, you will be able to input English sentence, and return the Deutch translation

In [1]:
from IPython.display import HTML
from subprocess import getoutput
s = getoutput('nvidia-smi')
print(s)
if 'K80' in s:
    gpu = 'K80'
elif 'T4' in s:
    gpu = 'T4'
elif 'P100' in s:
    gpu = 'P100'
else:
    gpu='DONT PROCEED'
display(HTML(f"<h1>{gpu}</h1>"))

Fri May  7 17:21:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install chart-studio

     |████████████████████████████████| 71kB 5.5MB/s 


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import pandas as pd
import unicodedata
import re
import numpy as np
import os
import io
import time
import string
from string import digits

import chart_studio.plotly
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
#%plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

##**Download and prepare the dataset**

We'll use a language dataset provided by http://www.manythings.org/anki/.


After downloading the dataset, here are the steps we'll take to prepare the data:


1. Add a start and end token to each sentence.
2. Clean the sentences by removing special characters.
3. Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
4. Pad each sentence to a maximum length.

In [4]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp /content/drive/MyDrive/deu-eng.zip ../

In [6]:
!unzip ../deu-eng.zip

Archive:  ../deu-eng.zip
  inflating: deu.txt                 
  inflating: _about.txt              


In [7]:
file_path = 'deu.txt'

In [8]:
lines = open(file_path, encoding='UTF-8').read().strip().split('\n')
lines[5000:5010]

['You promised.\tSie haben es versprochen.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2549737 (CK) & #3874405 (raggione)',
 'You scare me.\tDu beängstigst mich.\tCC-BY 2.0 (France) Attribution: tatoeba.org #1226152 (CK) & #2684672 (Pfirsichbaeumchen)',
 'You scare me.\tSie beängstigen mich.\tCC-BY 2.0 (France) Attribution: tatoeba.org #1226152 (CK) & #2684673 (Pfirsichbaeumchen)',
 'You scare me.\tDu machst mir Angst.\tCC-BY 2.0 (France) Attribution: tatoeba.org #1226152 (CK) & #2684676 (Pfirsichbaeumchen)',
 'You stay put.\tDu bleibst, wo du bist!\tCC-BY 2.0 (France) Attribution: tatoeba.org #2255244 (CK) & #2671956 (freddy1)',
 'You survived.\tSie haben überlebt.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2549735 (CK) & #4942912 (Hans_Adler)',
 'You survived.\tIhr habt überlebt.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2549735 (CK) & #4942913 (Hans_Adler)',
 'You survived.\tDu hast überlebt.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2549735 (CK) & #4942914 (Hans_Adler)

In [9]:
print("total number of records: ",len(lines))

total number of records:  227080


##**Clean and Preprocess the text**

1. Convert to lower case
2. Convert special characters
3. Remove Digits
4. Remove spaces
5. Add start and end tags to each sentence

In [10]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                 if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [11]:
en_sentence = u"How do you know what I'm thinking?"
deu_sentence = u"Woher wissen Sie, was ich denke?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(deu_sentence).encode('utf-8'))

<start> how do you know what i m thinking ? <end>
b'<start> woher wissen sie , was ich denke ? <end>'


##**Generate pairs of cleaned English and Deutch sentences with start and end added**

In [12]:
# Generate pairs of cleaned English and Deutch sentences
sent_pairs = []
#due to memory constraints we'd not be using the whole data (227080 sentences)
for line in lines[:150000]:                                          
    sent_pair = []
    eng = line.rstrip().split('\t')[0]
    deu = line.rstrip().split('\t')[1]
    eng = preprocess_sentence(eng)
    sent_pair.append(eng)
    deu = preprocess_sentence(deu)
    sent_pair.append(deu)
    sent_pairs.append(sent_pair)
sent_pairs[5000:5010]

[['<start> you promised . <end>', '<start> sie haben es versprochen . <end>'],
 ['<start> you scare me . <end>', '<start> du beangstigst mich . <end>'],
 ['<start> you scare me . <end>', '<start> sie beangstigen mich . <end>'],
 ['<start> you scare me . <end>', '<start> du machst mir angst . <end>'],
 ['<start> you stay put . <end>', '<start> du bleibst , wo du bist ! <end>'],
 ['<start> you survived . <end>', '<start> sie haben uberlebt . <end>'],
 ['<start> you survived . <end>', '<start> ihr habt uberlebt . <end>'],
 ['<start> you survived . <end>', '<start> du hast uberlebt . <end>'],
 ['<start> you ll do it . <end>', '<start> du wirst es tun . <end>'],
 ['<start> you re a spy . <end>', '<start> du bist ein spion . <end>']]

##**Create a class to map every word to an index and vice-versa for any given vocabulary.**

In [13]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

        self.create_index()

    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [14]:
def max_length(tensor):
    return max(len(t) for t in tensor)

##**Tokenization and Padding**

In [15]:
def load_dataset(pairs, num_examples):
    # pairs => already created cleaned input, output pairs

    # index language using the class defined above    
    inp_lang = LanguageIndex(en for en, de in pairs)
    targ_lang = LanguageIndex(de for en, de in pairs)
    
    # Vectorize the input and target languages
    
    # English sentences
    input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, de in pairs]
    
    # Deutch sentences
    target_tensor = [[targ_lang.word2idx[s] for s in de.split(' ')] for en, de in pairs]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [16]:
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(sent_pairs, len(lines))

##**Creating training and validation sets using an 80-20 split**

In [17]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1, random_state = 101)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(135000, 135000, 15000, 15000)

##**Create a tf.data dataset**

In [18]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [19]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 14]), TensorShape([64, 24]))

##**Define the encoder and decoder network**

In [20]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [21]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print('Encoder output shape: (batch size, sequence length, units)', sample_output.shape)
print('Encoder Hidden state shape: (batch size, units)', sample_hidden.shape)

Encoder output shape: (batch size, sequence length, units) (64, 14, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [22]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):

        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [23]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print('Decoder output shape: (batch_size, vocab size)', sample_decoder_output.shape)

Decoder output shape: (batch_size, vocab size) (64, 21754)


## **Define the optimizer and the loss function**

In [24]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction='none')


def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## **Checkpoints (Object-based saving)**

In [25]:
checkpoint_dir = '/content/drive/MyDrive/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## **Training**

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [26]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every epoch
    checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 3.0951
Epoch 1 Batch 100 Loss 1.5910
Epoch 1 Batch 200 Loss 1.4158
Epoch 1 Batch 300 Loss 1.3725
Epoch 1 Batch 400 Loss 1.3298
Epoch 1 Batch 500 Loss 1.2640
Epoch 1 Batch 600 Loss 1.1225
Epoch 1 Batch 700 Loss 1.1709
Epoch 1 Batch 800 Loss 1.2205
Epoch 1 Batch 900 Loss 1.1644
Epoch 1 Batch 1000 Loss 1.0894
Epoch 1 Batch 1100 Loss 1.1330
Epoch 1 Batch 1200 Loss 1.0825
Epoch 1 Batch 1300 Loss 1.0463
Epoch 1 Batch 1400 Loss 1.1150
Epoch 1 Batch 1500 Loss 0.9286
Epoch 1 Batch 1600 Loss 0.9807
Epoch 1 Batch 1700 Loss 0.9098
Epoch 1 Batch 1800 Loss 0.9225
Epoch 1 Batch 1900 Loss 0.9180
Epoch 1 Batch 2000 Loss 0.9068
Epoch 1 Batch 2100 Loss 0.8968
Epoch 1 Loss 1.1342
Time taken for 1 epoch 1630.48703789711 sec

Epoch 2 Batch 0 Loss 0.7836
Epoch 2 Batch 100 Loss 0.7737
Epoch 2 Batch 200 Loss 0.7651
Epoch 2 Batch 300 Loss 0.7377
Epoch 2 Batch 400 Loss 0.7031
Epoch 2 Batch 500 Loss 0.7534
Epoch 2 Batch 600 Loss 0.5929
Epoch 2 Batch 700 Loss 0.7148
Epoch 2 Batch 800 Loss 0.63

##**Inference and Testing**
* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [27]:
def evaluate(inputs, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = ''
    for i in inputs[0]:
        if i == 0:
            break
        sentence = sentence + inp_lang.idx2word[i] + ' '
    sentence = sentence[:-1]
    
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [28]:
def predict_random_val_sentence():
    actual_sent = ''
    k = np.random.randint(len(input_tensor_val))
    random_input = input_tensor_val[k]
    random_output = target_tensor_val[k]
    random_input = np.expand_dims(random_input,0)
    result, sentence, attention_plot = evaluate(random_input, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
    print('Input: {}'.format(sentence[8:-6]))
    print('Predicted translation: {}'.format(result[:-6]))
    for i in random_output:
        if i == 0:
            break
        actual_sent = actual_sent + targ_lang.idx2word[i] + ' '
    actual_sent = actual_sent[8:-7]
    print('Actual translation: {}'.format(actual_sent))
    attention_plot = attention_plot[:len(result.split(' '))-2, 1:len(sentence.split(' '))-1]
    sentence, result = sentence.split(' '), result.split(' ')
    sentence = sentence[1:-1]
    result = result[:-2]

    # use plotly to generate the heat map
    trace = go.Heatmap(z = attention_plot, x = sentence, y = result, colorscale='greens')
    data=[trace]
    iplot(data)

##**Restore the latest checkpoint and test**

In [29]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [36]:
predict_random_val_sentence()

Input: they need me .
Predicted translation: sie brauchen mich . 
Actual translation: sie brauchen mich .


In [37]:
predict_random_val_sentence()

Input: is it yours ?
Predicted translation: ist es dir ? 
Actual translation: ist es ihres ?


In [38]:
predict_random_val_sentence()

Input: i m glad you re safe .
Predicted translation: ich freue mich , dass du in sicherheit bist . 
Actual translation: ich bin froh , dass ihr in sicherheit seid .


In [39]:
import  nltk.translate.bleu_score as bleu

In [43]:
reference_translation='ist es ihres ?'.split()
candidate_translation='ist es dir ?'.split()

In [44]:
print("BLEU Score: ",bleu.sentence_bleu(reference_translation, candidate_translation))

BLEU Score:  0.7071067811865476


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning:


Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().



In [42]:
# 4-gram cumulative BLEU
from nltk.translate.bleu_score import sentence_bleu
reference = ' ich bin froh , dass ihr in sicherheit seid .'.split()
candidate = ' ich freue mich , dass du in sicherheit bist .'.split()
score = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))
print(score)

0.668740304976422


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning:


Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().

